DataJoint is a Python (or MATLAB) framework that allows manipulation of MySQL databases and tables within Python (or MATLAB).

Python (or MATLAB) classes to represent and link to MySQL tables. Manipulation of tables (create and drop tables, insert, delete and update entries) could be implemented by calling methods of table classes.

In [1]:
import datajoint as dj

In [2]:
# create schema, house of tables
schema = dj.schema('demo_nwb')

Connecting shan@localhost:3306


In [6]:
# create a table by declaring a class
@schema
class Mouse(dj.Manual):
    definition = """
    mouse_id       : int 
    ---
    dob            : date            # date of birth
    sex            : enum('M', 'F', 'unknown')
    """

In [7]:
Mouse()

mouse_id,dob date of birth,sex


In [8]:
# insert some mouse entries
data = [(0, '2019-08-25', 'M'),
        (1, '2019-10-21', 'F'),
        (2, '2020-01-01', 'unknown'),
        (3, '2019-02-03', 'M'),
        (4, '2019-08-23', 'F')
       ]

Mouse.insert(data)

In [9]:
Mouse()

mouse_id,dob date of birth,sex
0,2019-08-25,M
1,2019-10-21,F
2,2020-01-01,unknown
3,2019-02-03,M
4,2019-08-23,F


In [12]:
# create another table representing experimental session
@schema
class Session(dj.Manual):
    definition = """
    -> Mouse
    session_date       : date
    ---
    experimenter       : varchar(32)
    setup              : int        
    """

In [13]:
# insert some sessions
data = [
    {'mouse_id': 0, 'session_date': '2019-10-30', 'experimenter': 'Shan Shen', 'setup': 1},
    {'mouse_id': 0, 'session_date': '2019-11-02', 'experimenter': 'Shan Shen', 'setup': 1},
    {'mouse_id': 1, 'session_date': '2019-12-30', 'experimenter': 'Thinh Nguyen', 'setup': 2},
    {'mouse_id': 2, 'session_date': '2020-01-30', 'experimenter': 'Shan Shen', 'setup': 2},
]

In [14]:
Session.insert(data)

In [15]:
Session()

mouse_id,session_date,experimenter,setup
0,2019-10-30,Shan Shen,1
0,2019-11-02,Shan Shen,1
1,2019-12-30,Thinh Nguyen,2
2,2020-01-30,Shan Shen,2


In [16]:
Mouse & 'sex="M"'

mouse_id,dob date of birth,sex
0,2019-08-25,M
3,2019-02-03,M


In [17]:
Mouse & Session

mouse_id,dob date of birth,sex
0,2019-08-25,M
1,2019-10-21,F
2,2020-01-01,unknown


In [18]:
(Mouse & Session).fetch()

array([(0, datetime.date(2019, 8, 25), 'M'),
       (1, datetime.date(2019, 10, 21), 'F'),
       (2, datetime.date(2020, 1, 1), 'unknown')],
      dtype=[('mouse_id', '<i8'), ('dob', 'O'), ('sex', 'O')])

In [19]:
# delete entries
(Mouse & 'mouse_id=2').delete()

About to delete:
`demo_nwb`.`session`: 1 items
`demo_nwb`.`mouse`: 1 items
Proceed? [yes, No]: yes
Committed.


In [20]:
Mouse()

mouse_id,dob date of birth,sex
0,2019-08-25,M
1,2019-10-21,F
3,2019-02-03,M
4,2019-08-23,F


In [21]:
Session()

mouse_id,session_date,experimenter,setup
0,2019-10-30,Shan Shen,1
0,2019-11-02,Shan Shen,1
1,2019-12-30,Thinh Nguyen,2


In [22]:
# drop table
Mouse.drop()

`demo_nwb`.`mouse` (4 tuples)
`demo_nwb`.`session` (3 tuples)
Proceed? [yes, No]: yes
Tables dropped.  Restart kernel.
